<a href="https://colab.research.google.com/github/player-29/Data-Science-For-Beginners/blob/main/task_1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder  # or try OneHotEncoder if this doesnt work
import numpy as np
from torch.utils.data import TensorDataset
import torch.nn as nn
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

dataframe= pd.read_csv('/task_1a_dataset.csv')

dataframe

def data_preprocessing(task_1a_dataframe):
    dataset = task_1a_dataframe.copy()
    labelencoder = LabelEncoder()
    dataset['Education'] = labelencoder.fit_transform(dataset['Education'])
    dataset['City'] = labelencoder.fit_transform(dataset['City'])
    dataset['Gender'] = labelencoder.fit_transform(dataset['Gender'])
    dataset['EverBenched'] = labelencoder.fit_transform(dataset['EverBenched'])
    dataset['ExperienceInCurrentDomain'].fillna(dataset['ExperienceInCurrentDomain'].mean(), inplace=True)
    dataset['PaymentTier'].fillna(dataset['PaymentTier'].mean(), inplace=True)
    dataset['Age'].fillna(dataset['Age'].mean(), inplace=True)
    encoded_dataframe = dataset
    return encoded_dataframe

encoded = data_preprocessing(dataframe)
print(encoded)

encoded

def identify_features_and_targets(encoded_dataframe):
    features = encoded_dataframe.iloc[:, :-1]
    target = encoded_dataframe.iloc[:, -1]
    features_and_targets = [features, target]
    return features_and_targets

features_and_targets=identify_features_and_targets(encoded)
features_and_targets

import torch
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_sample = self.X[idx]
        y_sample = self.y[idx]
        return x_sample, y_sample

import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


def load__as__tensors(features_and_targets):
    features = features_and_targets[0]
    target = features_and_targets[1]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


    # Create custom dataset instances for training and testing
    train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
    test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

    # Create a DataLoader for training data
    batch_size = 64  # Adjust as needed
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    return [X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_dataloader]

tensors_and_iterable_training_data = load__as__tensors(features_and_targets)
tensors_and_iterable_training_data

tensors_and_iterable_training_data = load__as__tensors(features_and_targets)
tensors_and_iterable_training_data

encoded.drop('LeaveOrNot', axis=1).info()

encoded['LeaveOrNot'].info()

from torch.utils.data import DataLoader
X_train_tensor, X_test_tensor, y_train_tensor,y_test_tensor,train_dataloader = load__as__tensors(features_and_targets)
print("X_train_tensor shape:", X_train_tensor.shape)
print("y_train_tensor shape:", y_train_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)
print("y_test_tensor shape:", y_test_tensor.shape)
print("train_dataloader:",train_dataloader)

class Salary_Predictor(nn.Module):
    def __init__(self):
        super(Salary_Predictor, self).__init__()
        # Define your model architecture here
        self.fc1 = torch.nn.Linear(X_train_tensor.shape[1],128)
        self.fc2 = torch.nn.Linear(128,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x



model=Salary_Predictor()

def model_loss_function(): #can be in-built from pytorch, use binary cross entropy
  return nn.BCELoss()

loss_function=model_loss_function()

import torch.optim as optim
def model_optimizer(model):
  return torch.optim.Adam(model.parameters(), lr=0.01)

optimizer=model_optimizer(model)

def model_number_of_epochs():
  return 10

number_of_epochs=model_number_of_epochs()

def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):

	# Unpack tensors and iterable training data
	X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader = tensors_and_iterable_training_data;y_train_tensor = y_train_tensor.view(-1, 1)   ;y_test_tensor = y_test_tensor.view(-1, 1)

	for epoch in range(number_of_epochs):
		model.train()  # Set the model in training mode

		# Initialize variables to track loss and accuracy
		total_loss = 0.0
		correct_predictions = 0

		for batch_x, batch_y in train_loader:
			# Zero the gradients
			optimizer.zero_grad()

			# Forward pass
			outputs = model(batch_x);batch_y=batch_y.view(-1,1)
			# Calculate loss
			loss = loss_function(outputs,batch_y)

			# Backpropagate and update parameters
			loss.backward()
			optimizer.step()

			# Accumulate loss
			total_loss += loss.item()

			# Calculate the number of correct predictions
			_, predicted = torch.max(outputs, 1)
			correct_predictions += (predicted == batch_y).sum().item()

		# Calculate average loss and accuracy for the epoch
		avg_loss = total_loss / len(train_loader.dataset)
		accuracy = correct_predictions / len(train_loader.dataset)

		# # Print training progress

	return model

def validation_function(trained_model, tensors_and_iterable_training_data):

	 # Unpack tensors and validation data loader
	X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, test_loader = tensors_and_iterable_training_data

	trained_model.eval()  # Set the model in evaluation mode

	correct_predictions = 0
	total_examples = 0

	with torch.no_grad():  # Disable gradient computation for validation
		for batch_x, batch_y in test_loader:
			# Forward pass to make predictions
			outputs = trained_model(batch_x)


			# Calculate the number of correct predictions
			_, predicted = torch.max(outputs, 1)
			correct_predictions += (predicted == batch_y).sum().item()
			total_examples += batch_y.size(0)

	# Calculate the accuracy
	model_accuracy = correct_predictions / total_examples


	return model_accuracy


if __name__ == "__main__":

    # reading the provided dataset csv file using pandas library and
    # converting it to a pandas Dataframe
    task_1a_dataframe = pd.read_csv('/task_1a_dataset.csv')


    # data preprocessing and obtaining encoded data
    encoded_dataframe = data_preprocessing(task_1a_dataframe)

    # selecting required features and targets
    features_and_targets = identify_features_and_targets(encoded_dataframe)


    tensors_and_iterable_training_data = load__as__tensors(features_and_targets)
    # model is an instance of the class that defines the architecture of the model
    model = Salary_Predictor()

    # obtaining loss function, optimizer and the number of training epochs
    loss_function = model_loss_function()
    optimizer = model_optimizer(model)
    number_of_epochs = model_number_of_epochs()

    # training the model

    trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data,
                    loss_function, optimizer)

    # validating and obtaining accuracy
    model_accuracy = validation_function(trained_model,tensors_and_iterable_training_data)
    print(f"Accuracy on the test set = {model_accuracy}")

    X_train_tensor = tensors_and_iterable_training_data[0]
    x = X_train_tensor[0]
    jitted_model = torch.jit.save(torch.jit.trace(model, (x)), "task_1a_trained_model.pth")

      Education  JoiningYear  City  PaymentTier  Age  Gender  EverBenched  \
0             0         2012     0            3   37       1            0   
1             1         2017     1            2   28       1            0   
2             0         2017     1            2   36       1            0   
3             0         2015     0            3   27       1            1   
4             0         2017     0            3   29       1            0   
...         ...          ...   ...          ...  ...     ...          ...   
4628          0         2013     0            3   26       0            0   
4629          1         2013     2            2   37       1            0   
4630          1         2018     1            3   27       1            0   
4631          0         2012     0            3   30       1            1   
4632          0         2015     0            3   33       1            1   

      ExperienceInCurrentDomain  LeaveOrNot  
0                            